In [ ]:
#
# New Open AI 01 model
# 

using SparseArrays
using LinearAlgebra

# local modules
push!(LOAD_PATH, realpath("../src"))
# using lp_constants
# using lp_utils
using lp_problem
# using lp_standard_form_converter
using lp_read_mps
# using lp_revised_simplex

mps_folder_path = "../check/problems/mps_files/"

<details>
    <summary> First model </summary>

```julia
function revised_simplex(lp::LPProblem)
    println("Starting Revised Simplex Method")
    # If the problem is minimization, convert it to maximization by negating c
    if lp.is_minimize
        c = -lp.c
    else
        c = lp.c
    end

    # Copy A, b, vars, constraint_types
    A = lp.A
    b = lp.b
    vars = lp.vars
    constraint_types = lp.constraint_types

    # For now, only handle <= constraints
    if any(ct != '<' for ct in constraint_types)
        error("Only <= constraints are currently supported.")
    end

    # All variables are non-negative
    if any(lp.l .!= 0.0) || any(isfinite.(lp.u))
        error("Only non-negative variables with no upper bounds are currently supported.")
    end

    # Convert inequalities to equalities by adding slack variables
    m, n = size(A)
    A_aug = A
    c_aug = c
    vars_aug = vars

    for i in 1:m
        # Add slack variable
        slack_var = "s_$i"
        e = sparse(zeros(m))
        e[i] = 1.0
        A_aug = hcat(A_aug, e)
        c_aug = vcat(c_aug, 0.0)
        vars_aug = vcat(vars_aug, slack_var)
    end

    # Initialize basis with slack variables
    basic_indices = collect(n .+ (1:m))  # Convert to mutable array
    nonbasic_indices = collect(1:n)      # Convert to mutable array

    # Extract submatrices
    B = Matrix(A_aug[:, basic_indices])  # Basis matrix
    c_B = c_aug[basic_indices]

    # Start iterations
    max_iters = 100
    for iter = 1:max_iters
        println("\nIteration $iter")
        println("Current basis variables: ", vars_aug[basic_indices])
        # Compute basic feasible solution
        x_B = B \ b
        println("Current basic feasible solution x_B: ", x_B)

        # Compute simplex multipliers (dual variables)
        λ = c_B' * inv(B)

        # Compute reduced costs for non-basic variables
        reduced_costs = zeros(length(nonbasic_indices))
        for idx in 1:length(nonbasic_indices)
            j = nonbasic_indices[idx]
            a_j = A_aug[:, j]
            reduced_costs[idx] = c_aug[j] - λ * a_j
        end
        println("Reduced costs: ", reduced_costs)

        # Check for optimality
        if lp.is_minimize
            optimal = all(reduced_costs .>= -1e-8)
        else
            optimal = all(reduced_costs .<= 1e-8)
        end

        if optimal
            println("Optimal solution found.")
            # Construct full solution
            x = zeros(length(vars_aug))
            x[basic_indices] = x_B
            obj_value = dot(c_aug, x)
            if lp.is_minimize
                obj_value = -obj_value
            end
            # Return solution
            solution = Dict("Objective" => obj_value,
                            "Variables" => Dict(zip(vars_aug, x)))
            return solution
        end

        # Determine entering variable
        if lp.is_minimize
            entering_idx_rel = findmin(reduced_costs)[2]
        else
            entering_idx_rel = findmax(reduced_costs)[2]
        end
        entering_idx = nonbasic_indices[entering_idx_rel]
        entering_var = vars_aug[entering_idx]
        println("Entering variable: ", entering_var)

        # Compute direction d = B^{-1} * A_j
        a_j = A_aug[:, entering_idx]
        d = B \ a_j
        println("Direction d: ", d)

        # Determine leaving variable
        ratios = [d[i] > 1e-8 ? x_B[i]/d[i] : Inf for i in 1:length(d)]
        theta, leaving_idx_rel = findmin(ratios)
        if isinf(theta)
            error("Problem is unbounded.")
        end
        leaving_idx = basic_indices[leaving_idx_rel]
        leaving_var = vars_aug[leaving_idx]
        println("Leaving variable: ", leaving_var)
        println("Theta: ", theta)

        # Update basic indices
        basic_indices[leaving_idx_rel] = entering_idx
        nonbasic_indices[entering_idx_rel] = leaving_idx

        # Update B and c_B
        B[:, leaving_idx_rel] = A_aug[:, entering_idx]
        c_B[leaving_idx_rel] = c_aug[entering_idx]
    end

    error("Maximum iterations exceeded.")
end
```

</details>

## Refactored model for new LP Stuct

In [ ]:
function revised_simplex(lp::LPProblem)
    println("Starting Revised Simplex Method")

    # If the problem is minimization, convert it to maximization by negating c
    c = lp.is_minimize ? -lp.c : lp.c

    # Extract necessary components from the LPProblem struct
    A = lp.A
    b = lp.b
    vars = lp.vars
    constraint_types = lp.constraint_types
    l = lp.l
    u = lp.u

    # For now, only handle <= constraints
    if any(ct != 'L' for ct in constraint_types)
        error("Only <= constraints are currently supported.")
    end

    # Ensure all variables are non-negative and have no upper bounds
    if any(l .!= 0.0) || any(isfinite.(u))
        error("Only non-negative variables with no upper bounds are currently supported.")
    end

    # Convert inequalities to equalities by adding slack variables
    m, n = size(A)
    A_aug = A
    c_aug = c
    vars_aug = vars

    for i in 1:m
        # Add slack variable
        slack_var = "s_$i"
        e = sparse(zeros(m))
        e[i] = 1.0
        A_aug = hcat(A_aug, e)
        c_aug = vcat(c_aug, 0.0)
        vars_aug = vcat(vars_aug, slack_var)
    end

    # Initialize basis with slack variables
    basic_indices = collect(n .+ (1:m))  # Basic variables indices
    nonbasic_indices = collect(1:n)      # Non-basic variables indices

    # Extract basis matrix
    B = Matrix(A_aug[:, basic_indices])  # Basis matrix
    c_B = c_aug[basic_indices]

    # Start iterations
    max_iters = 100
    for iter in 1:max_iters
        println("\nIteration $iter")
        println("Current basis variables: ", vars_aug[basic_indices])

        # Compute basic feasible solution
        x_B = B \ b
        println("Current basic feasible solution x_B: ", x_B)

        # Compute simplex multipliers (dual variables)
        λ = c_B' * inv(B)

        # Compute reduced costs for non-basic variables
        reduced_costs = zeros(length(nonbasic_indices))
        for idx in 1:length(nonbasic_indices)
            j = nonbasic_indices[idx]
            a_j = A_aug[:, j]
            reduced_costs[idx] = c_aug[j] - λ * a_j
        end
        println("Reduced costs: ", reduced_costs)

        # Check for optimality
        optimal = lp.is_minimize ? all(reduced_costs .>= -1e-8) : all(reduced_costs .<= 1e-8)
        if optimal
            println("Optimal solution found.")
            x = zeros(length(vars_aug))
            x[basic_indices] = x_B
            obj_value = dot(c_aug, x)
            obj_value = lp.is_minimize ? -obj_value : obj_value

            # Return solution with variable names
            solution = Dict("Objective" => obj_value,
                            "Variables" => Dict(zip(vars_aug, x)))
            return solution
        end

        # Determine entering variable
        entering_idx_rel = lp.is_minimize ? findmin(reduced_costs)[2] : findmax(reduced_costs)[2]
        entering_idx = nonbasic_indices[entering_idx_rel]
        entering_var = vars_aug[entering_idx]
        println("Entering variable: ", entering_var)

        # Compute direction d = B^{-1} * A_j
        a_j = A_aug[:, entering_idx]
        d = B \ a_j
        println("Direction d: ", d)

        # Determine leaving variable
        ratios = [d[i] > 1e-8 ? x_B[i] / d[i] : Inf for i in 1:length(d)]
        theta, leaving_idx_rel = findmin(ratios)
        if isinf(theta)
            error("Problem is unbounded.")
        end
        leaving_idx = basic_indices[leaving_idx_rel]
        leaving_var = vars_aug[leaving_idx]
        println("Leaving variable: ", leaving_var)
        println("Theta: ", theta)

        # Update basic indices
        basic_indices[leaving_idx_rel] = entering_idx
        nonbasic_indices[entering_idx_rel] = leaving_idx

        # Update B and c_B
        B[:, leaving_idx_rel] = A_aug[:, entering_idx]
        c_B[leaving_idx_rel] = c_aug[entering_idx]
    end

    error("Maximum iterations exceeded.")
end


In [ ]:
mps_filename = mps_folder_path * "ex_9-7.mps"
lp = read_mps_from_file(mps_filename)
print(lp)

In [ ]:
revised_simplex(lp)